In [12]:
import pandas as pd
import altair as alt
import wbgapi as wb

In [3]:
happy_all = pd.read_csv("./data/world-happiness-report.csv")
happy_21 = pd.read_csv("./data/world-happiness-report-2021.csv")

happy = pd.merge(happy_all[happy_all['year'] == 2019],
                  happy_21[['Country name', "Regional indicator" ]],
                  on = "Country name",
                  how  = "inner",
                 indicator = True)

In [83]:

alt.themes.enable('urbaninstitute')
#latimes
#ggplot2

# Definir un tema, más sobre esto: 
#https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602
def tema_graf():
    font = "Helvetica"
    labelFont = "Garamond"
    sourceFont = "Calibri"
    
    return {
        "config" : {
             "title": {'font': font, 
                       'fontSize': 15,
                       'fontWeight': 300,
                       'offset':0,
                       'anchor':'left'},
             "axis": {
                  "labelFont": font,
                  "titleFont": font,
                  "domainColor": "gray"
             },
             "header": {
                  "labelFont": font,
                  "titleFont": font
             },
             "legend": {
                  "labelFont": labelFont,
                  "titleFont": labelFont,
                  "symbolType": 'circle',
                  "padding": 1
                  
             }
             ,
             'background': '#47e6e6'             
        }
    }

#alt.themes.register('mitema', tema_graf)
#alt.themes.enable('mitema')

In [88]:
# Poniendo título a nuestro gráfico 
alt.Chart(happy, height = 500, width = 800).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator"),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Red"
    })

alt.Chart(...)

In [9]:
#Cambiando los titulos de los ejes

alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q",  title = "índice de felicidad"),
    y = alt.Y("Log GDP per capita:Q" ,  title = "PBi per capita (logs)"),
    color = alt.Color("Regional indicator"),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Red"
    })

alt.Chart(...)

In [11]:
#Cambiando los titulos de los ejes
## Para ver los palettes disponibles https://vega.github.io/vega/docs/schemes/
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q",  title = "índice de felicidad"),
    y = alt.Y("Log GDP per capita:Q" ,  title = "PBi per capita (logs)"),
    color = alt.Color("Regional indicator", scale=alt.Scale(scheme='tableau10')),
    size = alt.Size('Generosity')
    ).interactive().properties(
    title={
      "text": ["Esto parece un gráfico inspirado en Rosling"], 
      "subtitle": ["Es un homenaje",
                   "Fuente: Happiness Report"],
      "color": "Black",
      "subtitleColor": "Red"
    })

alt.Chart(...)

In [4]:
#happy_peru = happy_all[happy_all['Country name'] == 'Peru']

In [147]:
## Usando la api para descargar información del Banco mundial
## pip install wbgapi
## si no puedes usar pip, prueba hacer conda install pip
## Fuente: https://stackoverflow.com/questions/41060382/using-pip-to-install-packages-to-anaconda-environment

tbl = wb.data.DataFrame('SP.POP.TOTL',
                  time = range(2000, 2019), labels = True)

tbl = tbl.reset_index()

tbl.rename(columns = {'SP.POP.TOTL': 'poblacion'}, inplace = True)


lst = wb.economy.info().table()
lst.pop()
nombres = pd.DataFrame(data = lst, columns = ["economy", "Country", "Region", "income_classification"])
nombres.head(5)

tbl = pd.merge(tbl, nombres, on = ["economy", "Country"], how = "inner")

In [148]:
tbl = tbl.loc[tbl['Region'] != '']

In [149]:
tbl_melt = tbl.melt( id_vars = ['economy', 'Country', 'Region', 'income_classification'],
                   value_vars = [col for col in tbl if col.startswith('YR')])

In [150]:
tbl_melt['year'] = tbl_melt['variable'].str.replace('YR', '').astype('int')

tbl_melt.head(10)

,economy,Country,Region,income_classification,variable,value,year
0,ZWE,Zimbabwe,SSF,LMC,YR2000,11881477.0,2000
1,ZMB,Zambia,SSF,LMC,YR2000,10415944.0,2000
2,YEM,"Yemen, Rep.",MEA,LIC,YR2000,17409072.0,2000
3,PSE,West Bank and Gaza,MEA,LMC,YR2000,2922153.0,2000
4,VIR,Virgin Islands (U.S.),LCN,HIC,YR2000,108642.0,2000
5,VNM,Vietnam,EAS,LMC,YR2000,79910412.0,2000
6,VEN,"Venezuela, RB",LCN,UMC,YR2000,24192446.0,2000
7,VUT,Vanuatu,EAS,LMC,YR2000,184972.0,2000
8,UZB,Uzbekistan,ECS,LMC,YR2000,24650400.0,2000
9,URY,Uruguay,LCN,HIC,YR2000,3319736.0,2000


In [175]:

## agregaciones, y otros gráficos

g1 = alt.Chart(tbl_melt).mark_area().encode(
    x="year",
    y="sum(value):Q",
    color="Region:N"
)

g1

alt.Chart(...)

In [152]:
peru = tbl_melt.loc[tbl_melt['Country'] == 'Peru']
peru.head()

,economy,Country,Region,income_classification,variable,value,year
62,PER,Peru,LCN,UMC,YR2000,26459944.0,2000
279,PER,Peru,LCN,UMC,YR2001,26799285.0,2001
496,PER,Peru,LCN,UMC,YR2002,27100968.0,2002
713,PER,Peru,LCN,UMC,YR2003,27372226.0,2003
930,PER,Peru,LCN,UMC,YR2004,27624213.0,2004


In [173]:
g2 = alt.Chart(tbl_melt).mark_bar().encode(
    x = alt.X("year"),
    y="value:Q",
    color=alt.condition(
        alt.datum.year == 2005,
        alt.value("red"),  # The positive color
        alt.value("blue")  # The negative color
    )
)

In [177]:
g3 = alt.Chart(tbl_melt).mark_area().encode(
    x='year:Q',
    y='sum(value):Q',
    color='Region:N',
    row = alt.Row('Region:N')
).properties(height=50, width=800)

g3

alt.Chart(...)

In [176]:
alt.hconcat(g1, g2)

alt.HConcatChart(...)

In [178]:
alt.vconcat(alt.hconcat(g1, g2), g3)

alt.VConcatChart(...)